- `print()`는 느리다. 

- `input()`도 많이 느리다.
  - 입력이 많을것으로 예상되는 경우, `input=sys.stdin.readline`을 사용한다.
  - 메모리가 문제가 되지 않는 경우 `input=io.BytesIO(os.read(0, os.fstat(0).st_size)).readline`을 사용할 수 있다.
    - standard input으로 입력 파일의 크기(`os.fstat(0).st_size`)만큼 까지 file descriptor에서 직접 bytes로 읽어들인다.
    - `os.read(0, os.fstat(0).st_size)` 에 의해 모든 스탠다드 입력 스트림이 버퍼안으로 들어간다
    - `~.readline`은 개행문자를 찾을 때까지 스트림을 읽어온다.
    - 읽어온 스트림은 bytes 타입이므로, 다른 string을 비교해야 할 경우 bstring 변환, 혹은 인코딩하여 비교해야 한다.

- CPython에서 전역변수에 접근하는 시간은 로컬변수에 접근하는 시간보다 유의미하게 느리다.
  - 따라서 가능한 경우, 전역 범위에 변수를 선언하는것보단, \
  문제를 푸는 함수(`def sol(): ...`)를 선언해서 그 TC 안에서 필요한 변수들을 사용하는 것이 좋다.

- 파이썬의 index 접근은 생각보다 느리다. 느린 $O(1)$이다.
  - 예를들어 list `l`을 index인 `i`로 접근하려 할 때, \
  `for i in range(len(l)): l[i]`보다, `for v in l: v`가 더 빠르며, \
  index가 필요할 경우 `for i, v in enumerate(l): i, v`를 사용하는 것이 더 빠르다.
  - 특히 2차원 이상의 배열에서 (바뀌지 않을)원본 데이터를 읽어야 할 경우, `range`를 두번 써서 인덱스 접근을 하게 되는 경우 \
  예를들어,  `for i in range(N): for j in range(N): L[i][j]` 인터프리터는 L[i][j]의 값을 알아내기 위해 매번 `L[i][j]`에 접근해야 한다. \
  `for i, l in enumerate(L): for j, v in enumerate(l): v`의 경우 v의 값은 캐싱되어 있으므로 상대적으로 더 느린 인덱스 접근을 할 필요가 없어진다. \
  이를 통해 평균 약 10%정도 동작속도가 빨라진다.
  - 하지만 list의 길이를 동적으로 두어 append 하는 것(`l.append(x)`)보다, \
  i가 충분히 클 경우 l의 길이를 fixed 해놓고, `l[i] = x`를 사용하는 것이 더 빠르다.
    - https://www.acmicpc.net/source/61132478 `l[i] = x` 구현. 492ms
    - https://www.acmicpc.net/source/61132338 `l.append(x)` 구현. 540ms
  - DP에 저장할 값을 tuple이나 list로 하는 것은 속도를 느리게 할 수 있다. 9019(DSLR) 참고
    - 값이 2개 이상 필요한 경우 DP 테이블을 하나 더 만들어서 각각 저장하는 것이 더 빠르다.


- Python의 String은 immutable 하기 때문에, 합쳐진 문자열을 만들기 위해선 원본 문자열들의 모든 문자를 복사하는 작업을 해야 하므로 $O(N^2)$ 만큼의 시간이 소요된다.
  - 쉽게 생각해서, `string` += `string` 같은 표현을 절대 삼가자. 이 과정이 $O(N)$ 이다. \
  대신 `string`을 list로 관리하고, 그 list에 `append()`하는 방식을 사용하자. 나중에 문자열을 합칠 때 `join()`을 사용해서 한꺼번에 하는 것이 효율적이다.


- list(deque)나 tuple형태로 저장하고 꺼내쓰는 방식은 매우 비효율적이고, 느리다.
  - Python은 추가 메모리 할당을 최소화 하기 위해 메모리를 넉넉하게 잡아둔다. 반복문으로 많은 원소를 append를 하게 되면 결국 오래 걸리는 추가 메모리 할당을 해야 하며, 이는 런타임에 진행된다.
    - Python이 메모리를 필요 이상으로 사용하는 것도 이러한 이유
  - list를 만드는 작업 자체도 오래 걸린다. immutable한 값을 그대로 넣는 것보다, 오래걸리는 메모리 할당을 필요로 하는 게 당연히 시간이 더 걸린다.
  - Garbage Collector도 문제가 된다. q에서 list가 빠져나왔을 때, GC는 빠져나온 list를 할당 해제 할지의 여부를 런타임에 확인한다.
    - 즉, (확인하는 시간) + (할당 해제하는 시간)이 런타임에 소모된다.
  - 따라서 가능하면 특히, node는 list로 저장하는 것을 피해야 한다. 분리할 수 있으면 각각 나눠서 처리하자.


- Python은 재귀 함수 최적화가 없다. 아랫 문단 참조.

- `if~ else` 문을 사용할 때, 가능하면 마지막 case는 else로 처리하는 것이 빠르다. 
  - 프로그램 입장에서 언제 해당 블럭이 끝나는지 알 수 있게되어 최적화가 된다.

### PyPy 최적화 테크닉
- https://www.pypy.org/performance.html
- PyPy는 JIT 컴파일러를 채택했기 때문에 CPython을 구현한 Python과 비교하여, 성능 차이와 최적화 적용 여부가 사뭇 다르다. 관련한 최적화 테크닉을 기술한다.
- 소스 코드가 simple해야 한다(Keep it simple)
  - PyPy JIT은 자주 쓰이는 사용 패턴에 대해서 최적화를 적용한다. 따라서 코드가 복잡하면 최적화가 적용되지 않을 수 있다.\
  시간 복잡도를 해치지 않는 선에서 코드를 간결하게 작성하면 속도면에서 이득을 볼 수 있다.
- 전역 변수의 사용이 문제되지 않는다.
  - CPython에선 전역변수 사용(더 나아가 outer scope에 존재하는 변수들) 및 패키지 import가 local변수를 사용하는 것 보다 느리다. 
    - 예를들어, 블럭 단위에서도, 어떤 int()변수를 자주 사용한다면 그 블럭 내부에 선언하면 효과를 볼 수 있다.
  - PyPy에선 이러한 내용이 해당되지 않는다. PyPy JIT 관점에선 그냥 extra copy를 만드는 것에 불과.
  - 다만, 모듈성을 위해서 여전히 local 변수를 복사해 두는 것은 좋다.
- regex 패턴 search/match/replation 연산이 비싸다.
- `Old- vs. new-style classes` 이부분은 Python 3.x 이후 모든 class가 new-style을 사용하게 되어있으므로 무시해도 좋다.
- 느린 문자열의 concatenation. 상기해둔 내용과 같지만, 좀더 구체적인 내용을 적는다.
  - `for x in mylist: s += foo(x)` 와 같이 적는것은 시간 복잡도가(O(N^2)$ 이고, 메모리가 mylist의 quadratic만큼 소모된다. 
    - s는 매번 x를 합친 훨씬 큰 접두사를 복사해야 하기 때문
    - 단, s가 bytearray일 경우 in-place에서 작동하므로 이러한 문제가 발생하지 않는다.
  - `"".join(s)` 등의 구현은 단 한개의 문자열 객체를 생성하고, 단 한번의 C언어 단의 sequence 복사가 이뤄지기에 훨씬 빠르다.


### 파이썬의 재귀, 비재귀화
- 파이썬은 재귀함수 최적화가 전혀 없다.
  - parameter로 전달되는 값들은 참조 값이 전달되는 것도 아닌 값의 복사본이다.
  - 심지어 참조값의 id조차도 계속해서 복사되기 때문에 메모리가 엄청나게 든다.
  - 최적화가 없다는 의미는, 이러한 복사를 전부 런타임에서 진행한다는 점이고, \
  재귀함수 call의 오버헤드가 다른 언어보다 클 수 밖에 없다.
- Python 개발자들은 이에 대한 최적화는 앞으로도 없을 것이라고 못 박았다.(출처는 몰루) \
재귀문이 흔한 PS판에서 파이썬으로 살아남으려면, 처세를 잘 해야한다. 아래에서 대처법을 잘 찾아보자.

#### 왜 느린가?
- Recursion Depth Limit의 존재, 그리고 그 기본값이 낮음.
  - 결과론적인 내용이지만, 기본 재귀 싶이 제한이 1000으로 낮다는 것을 일종의 증거로 보일수도 있을 것 같다. 물론 PS의 관점에서만.
  - 기본 재귀재한인 1000으로 풀 수 있는 PS 문제는 적다(사실, 그래프 문제가 아니라면 그렇게 적은건 아니다). \
  이렇게 낮은 수치로 책정된 이유는 재귀함수가 그만큼 비효율적이라는 증거가 될 수도 있다.
- 재귀함수를 호출한다는 것을 `함수 호출을 여러번 한다`는 것에 집중해보자. \
그렇다면, 파이썬은 함수를 호출 때 얼만큼의 오버헤드가 발생하는 것일까?
  - Type Checking의 필요: 파이썬은 동적 타입 언어이다. \
  잘 생각해보면 Python에서 return할 값은 기본적으로 `None`이고, 우리가 값을 원할 때 `int` 타입 등의 값을 반환한다.\
  그것 외에도, PS에선 잘 쓰이지 않지만 함수 내부의 구현에 따라 반환할 타입이 여러개가 될 수 있다.\
  즉, 매 재귀마다 Type도 확인해야한다는 것이다. 여기서 오버헤드가 발생한다. PS판에선 좀 억울할 듯.
    - 참고로 PyPy에서도 이러한 사항은 예외가 아니다. 
    - https://doc.pypy.org/en/latest/faq.html#would-type-annotations-help-pypy-s-performance
  - *args, **kwargs의 처리: 파이썬에서는 함수의 parameter를 `*args, **kwargs`로 받을 수 있다. \
  arg랑 kwargs가 없을때의 최적화는 존재하겠지만, 만약 이런 값을 다룬다면 list와 dict를 만드는 오버헤드도 발생한다.
  - PyPy 와도 공유하는 특징으로 GIL(Global Interpreter Lock)이 있다.\
  간단하게 말해, 파이썬이 사용하는 객체의 Thread-Safe를 보장하기 위한 일종의 보호장치이다.\
  파이썬의 GC는 오브젝트를 참조의 참조 횟수 기반으로, 이 횟수가 0이된 객체체의 메모리를 해제하는 방식으로 작동한다.\
  모든 객체에 mutex가 걸려있진 않겠지만, 적어도 그 GC를 위한 참조 횟수를 변경하는 오버헤드가 존재한다는 의미가 된다.
    - https://it-eldorado.tistory.com/160
    - https://doc.pypy.org/en/latest/faq.html#does-pypy-have-a-gil-why
- 스택 프레임 할당에 필요한 공간이 많다.\
쉽게 말해서, 파이썬의 함수는 체급이 크다는 뜻이다. 체급이 큰 걸 양산하려 했으니 비용이 많이드는 것은 당연하다.
  - 파이썬에서 스택 프레임에 포함되는 내용들을 논리적으로 구분한다면 다음과 같다.
  - 지역변수: 함수 내부에서 변수가 선언될 수 있다. 이 변수들은 스택 프레임에 저장된다.\
  문제는, 함수의 parameter 또한 이 지역변수에 포함된다는 것이다.\
  따라서 재귀함수의 경우, parameter로 전달되는 값들이 계속해서 복사되어야 한다. \
  그러한 behavior를 가져야 하는 것이 재귀함수 이기도 하고.
    - 함수의 parameter가 많을 수록 오버헤드가 커진다는 점을 여기서 유도할 수 있다.
    - 이에따라, 재귀 함수에서 전역적으로 변하는 부분을 paramter에서 빼는 방법으로 개발자는 약간의 최적화를 시도할 수 있다.
    - PS 관점에서, 세그먼트 트리를 재귀로 짠 코드를 생각해보자. \
    update()는 (l, r, s, e, x)로 5개나 필요한데,\
    이게 얼마나 Python에서 비효율적으로 작동할지 직관적으로 가늠할 수 있다.
  - return value: 반환값 그 자체도 스택 프레임에 포함될 수 밖에 없다.
  - 이전 스택 프레임의 포인터: 함수가 호출될 때마다, 이전 스택 프레임의 포인터도 저장해야 한다.
- inlining 등의 Code transformation 의 부재
  - 컴파일러가 `최적화`를 한다는 일반적인 개념에서 할 법한 행위이다.\
  예를들어, 어떤 간단한 함수가 있다고 하자. `def f(x): return x+1`\
  이 함수를 호출하는 코드가 여러번 있다면, 컴파일러는 이 함수를 호출하는 코드를 `x+1`로 치환할 수 있다.\
  이 부분을 컴파일러가 어셈블리나, 바이트코드를 작성하는 과정에서 최적화 한다면 이것을 보고 inlining이라고 할 수 있다.
    - https://en.wikipedia.org/wiki/Inline_expansion
  - 이 부분은 CPython에서 좀 더 부각된다. \
  CPython은 기본적으로 inlining 최적화를 수행하려하지 않는다. 즉, 코드가 있는 그대로 실행된다는 뜻이다.\
  Python 3.10부터 이러한 악명높은 function call에 대한 최적화가 진행되고 있지만, \
  기본적으로 프로그램에 대한 최적화가 들어간다고 보긴 어렵다.
    - https://softwareengineering.stackexchange.com/questions/441670/is-code-written-inline-faster-than-using-function-calls
  - PyPy에선 약간 다른 이야기다. PyPy는 JIT 컴파일러를 채택했고, \
  당연하게도 코드를 변형하는 수준의 다양한 최적화(inlining, unrolling, 휴리스틱 등)를 수행한다.\
  또한, 위의 문단에서 설명한 common pattern optimization도 어떤 면에선 inlining의 확장이라고도 볼 수 있다.
  그렇다면, 이론적으로 재귀 함수 패턴에 대해서도 적용될 수 있다는 뜻이다.\
  그래서, 진짜로 빨라지는가? 전혀 아니다. \
  PyPy로 재귀 함수를 짜면 가끔 정답을 맞을 것도 TLE, MLE가 나곤 한다.\
  슬프게도, PS로 재귀문을 짠다면 JIT의 구원을 받을 수 없는듯 하다. \
  최적화 대상이 될 재귀문에 대한 common pattern이 없는 것인지, \
  아니면 애초에 PS가 common pattern이 아닌 것을 코딩하는 것인지는 모르겠다.
    - PyPy JIT 옵션: https://doc.pypy.org/en/latest/jit_help.html
    - 애초에 PyPy가 CPython보다 빠를 수 있는 메이저한 이유가 이거다.
  - CPython과 PyPy는 서로 다른 이유로써 재귀함수가 느리다.
- Tail-Call Optimization의 부재
  - 특수한 구조의 재귀함수에 대해서 컴파일러가(Python의 관점에선 Python 구현체가) 수행하는 재귀 최적화이다.
    - 이를 파이썬 문법으로 짠다면 대충 다음과 같은 형태일 때 TCO가 적용된다.
    - `def F(x, acc=1): return F(x-1, acc*x) if x > 0 else acc`
  - 어짜피 PS에서도, 현업에서도 잘 쓰지도 않는 테크닉이다만, (아마 FOP에선 좀 쓰일지도) \
  Python과 PyPy는 거기에 한술 더 떴다. 아예 TCO를 지원을 안한다.\
  따라서 자세히 다루지도 않겠다.

#### sys.setrecursionlimit()
- 내가 이 문서를 작성할 시점에서도 가장 널리 알려진 해결 방법.
- 재귀함수의 call stack을 늘려주는 방법이다. 기본값은 `1000`이고, 이 값을 늘림으로써 재귀함수를 더 깊게 호출할 수 있다.
- 사실, 이 방법은 속도 문제를 더 악화시킨다.
  - 이미 파이썬의 재귀함수가 비효율적인데, 이를 더 깊게 호출하면 더 비효율적일 수 밖에 없다.
  - 특히 PyPy랑 같이 쓸 때 큰 문제가 되는데, PyPy의 자세한 구현을 살펴본 것은 아니지만, \
  recursion limit 을 높게 설정하는 것 만으로도 수행시간과 메모리 소모에 영향을 주는 것 같다.
    - JIT의 최적화가 심각하게 영향을 받는게 원인일 듯 하다.
  - 이 limit에 대한 속도 체감은, 경험적으로 지수적이다.\
  즉, 재귀 깊이 제한이 높다면, 선형 수준이 아닌 지수적 수준으로 속도가 느려진다는 것이다.
    - 재귀 함수의 호출을 할 때 parameter로 넘어가는 값들이 복사되고, 
- 알려진 풀이가 재귀 함수밖에 없고, 다른 방법을 고민할 시간과 여유가 없다면, 이 방법밖에 없을 것이다.

#### 비재귀화
- 

### 파이썬의 시간복잡도 참고자료
- https://wiki.python.org/moin/TimeComplexity